In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs

In [2]:
url = 'https://movie.naver.com/movie/running/current.naver'
page = urlopen(url)
soup = bs(page, 'lxml')
soup.title.text

'현재 상영영화 : 네이버 영화'

In [13]:
# 초기 정보 필터
# 상영작/예정작 제목만 뽑기
soup_ul_li = soup.find("ul", class_="lst_detail_t1").find_all("li")

In [8]:
# 영화 개수 확인
print(len(soup_ul_li))
# 1번째 영화 제목 출력 확인
print(soup_ul_li[0].find("dt", class_="tit").a.text)
# 5번째 영화 제목 출력 확인
print(soup_ul_li[4].find("dt", class_="tit").a.text)
# 73번째(마지막) 영화 제목 출력 확인
print(soup_ul_li[-1].find("dt", class_="tit").a.text)

74
아이를 위한 아이
스파이형 모델
레베카


In [11]:
# 평점
# 출력 확인
print(soup_ul_li[0].find("span", class_="num").text)

# 참여 명수
# 출력 확인
print(soup_ul_li[0].find("em").text)

# 예매율
# print(soup_ul_li[0].find("dl", class_="info_exp").span.text)
temp = soup_ul_li[-1].find("dl", class_="info_exp")
if temp is not None:
	t = temp.span.text
	print("값이 있음.", t)
else:
	t = 0
	print("값이 없음.", t)

9.19
31
값이 없음. 0


In [18]:
# 개요
print(soup_ul_li[1].find("span", class_="link_txt").text)
txt = soup_ul_li[1].find("span", class_="link_txt").text
txt_last = txt.replace("\n", "")
txt_last = txt_last.replace("\t", "")
txt_last = txt_last.replace("\r", "")
print(txt_last)


액션, 
								
									판타지, 
								
									SF

액션, 판타지, SF


In [20]:
# 2-3 (추가) 기타 정보(감독, 출연, 상영시간) 추가 후 - 댓글 달기
# 감독
director = soup_ul_li[0].find("dl", class_="info_txt1").find_all("dd")[1].text
print(director)

# 출연
actor = soup_ul_li[10].find("dl", class_="info_txt1").find_all("dd")[-1].text
print(actor)



이승환




오스틴 버틀러, 
								
									톰 행크스




In [21]:
lst = []
for one in soup_ul_li:
	actor = one.find("dl", class_="info_txt1").find_all("dd")[-1].text
	txt_a = actor.replace("\n", "")
	txt_a = txt_a.replace("\t", "")
	txt_a = txt_a.replace("\r", "")
	lst.append(txt_a)
print(len(lst))

74


In [22]:
# 상영시간(...)
info_set = soup_ul_li[0].find("dl", class_="info_txt1").find_all("dd")[0].children
time = list(info_set)[-3].strip()
print(info_set)
print(time)
time_lst = []
for i in soup_ul_li:
	time = list(i.find("dl", class_="info_txt1").find_all("dd")[0].children)[-3].strip()
	time_lst.append(time)
print(len(time_lst))

96분
74


## 코드 정리
* 제목, 평점, 참여수, 개요, 감독, 출연

In [23]:
all_title = []
all_score = []
all_people = []
all_rate = []
all_category = []
all_director = []
all_actor = []
all_time = []

for one in soup_ul_li:
	# 제목, 평점, 참여수 텍스트화
	title = one.find("dt", class_="tit").a.text
	score = one.find("span", class_="num").text
	num_people = one.find("em").text

	# 예매율(웹사이트ㅔ서 없어짐.)
	tmp = one.find("dl", class_="info_exp")
	if tmp is not None:
		rate = tmp.span.text
	else:
		rate = 0

	# 카테고리 빈공간 정리
	category = one.find("span", class_="link_txt").text
	txt_last = category.replace("\n", "")
	txt_last = txt_last.replace("\t", "")
	txt_last = txt_last.replace("\r", "")

	# 감독 빈공간 정리
	director = one.find("dl", class_="info_txt1").find_all("dd")[1].text
	txt_d = director.replace("\n", "")
	txt_d = txt_d.replace("\t", "")
	txt_d = txt_d.replace("\r", "")

	# 상영 시간 텍스트 화
	time = list(one.find("dl", class_="info_txt1").find_all("dd")[0].children)[-3].strip()

	# 출연 빈공간 정리
	actor = one.find("dl", class_="info_txt1").find_all("dd")[-1].text
	txt_a = actor.replace("\n", "")
	txt_a = txt_a.replace("\t", "")
	txt_a = txt_a.replace("\r", "")

	all_title.append(title)
	all_score.append(score)
	all_people.append(num_people)
	all_rate.append(rate)
	all_category.append(txt_last)
	all_director.append(txt_d)
	all_actor.append(txt_a)
	all_time.append(time)

print(len(all_title),len(all_score), len(all_people),len(all_rate), len(all_category), len(all_director), len(all_actor), len(all_time))

74 74 74 74 74 74 74 74


In [24]:
import pandas as pd

dat_dict = {
	"제목" : all_title, "평점" : all_score, "참여 명수" : all_people, "예매율" : all_rate, "개요" : all_category, "감독" : all_director, "출연" : all_actor, "상영시간" : all_time
}
dat = pd.DataFrame(dat_dict)

In [25]:
dat.shape

(74, 8)

In [26]:
dat.head()

,제목,평점,참여 명수,예매율,개요,감독,출연,상영시간
0,아이를 위한 아이,9.19,31,0,,이승환,"현우석, 박상훈, 정웅인",96분
1,외계+인 1부,7.30,"9,928",0,"액션, 판타지, SF",최동훈,"류준열, 김우빈, 김태리, 소지섭, 염정아, 조우진, 김의성",142분
2,미니언즈2,7.93,"1,186",0,"애니메이션, 모험, 코미디",카일 발다,"스티브 카렐, 타라지 P. 헨슨, 양자경, 루시 로리스, 장 끌로드 반담",87분
3,썸머 필름을 타고!,7.77,120,0,"멜로/로맨스, SF",마츠모토 소우시,"이토 마리카, 카네코 다이치, 이노리 키라라, 카와이 유미",98분
4,스파이형 모델,7.75,12,0,SF,김영일,"주종혁, 이아주, 장현동, 이병수, 박진수",86분
